## NextFLAP Example

This python notebook shows the usage of NextFLAP within unified planning library.

[![Open In GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https://github.com/aiplan4eu/up-nextflap/blob/master/example/NextFLAP.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aiplan4eu/up-nextflap/blob/master/example/NextFLAP.ipynb)


In particular we will go through the following steps:

 - create a numeric and temporal planning problem;
 - call the planner to solve the problem;


# Setup

First, we install up_nextflap library and its dependencies from PyPi. Here, we use the `--pre` flag to use the latest development build.

In [ ]:
!pip install --pre up_nextflap

### Imports
The basic imports we need for this demo are abstracted in the `shortcuts` package.

In [2]:
import unified_planning as up
from unified_planning.engines import PlanGenerationResultStatus
from unified_planning.shortcuts import (Problem, UserType, Object, Variable,
                                        BoolType, RealType,
                                        StartTiming, EndTiming, ClosedTimeInterval, Timing,
                                        Equals, GE, GT, LT, LE, Not,
                                        Plus, Minus, Times, Div,
                                        Forall)
from unified_planning.model.timing import Timepoint, TimepointKind

##Importing and registering NextFLAP

We register NextFLAP as a planning engine on the UPF platform:

In [3]:
from up_nextflap import NextFLAPImpl

env = up.environment.get_environment()
env.factory.add_engine('nextflap', __name__, 'NextFLAPImpl')

### Problem representation

In this example, we are going to model a problem in which a robot has to fill several water tanks up to a certain limit.

#### Types

The first thing to do is to introduce the required types. In our case, we will define the types `robot` and `location`:

In [4]:
robot = UserType('robot')
location = UserType('location')

## Fluents

Now we define the problem and define the fluents we need:
*   `at-robot`: with two parameters (`robot` and `location`), it models the position of the robot.
*   `at-station`: with one parameter (`location`), it models the locations that have a charging station. At these stations, the robots can recharge their batteries.
*   `attached-to-station`: with two parameters (`robot` and `location`), it indicates when a robot is docked to the station. The battery can only be recharged while the robot is docked.
*   `at-tap`: with one parameter (`location`), it models the locations that have a tap. At these locations, the robots can fill their jars with water.
*   `at-tank`: with one parameter (`location`), it models the locations where there is a water tank.
*   `tank_ready`: with one parameter (`location`), it represents that the water tank located at this location has reached the desired level of water. There can only be one tank at a given location. Once the tank is ready, its level can no longer be modified.
*   `battery-level`: with one parameter (`robot`), it models the battery level of the robot (real number from 0% to 100%).
*   `jar-level`: with one parameter (`robot`), it models the amount of water (real number) the robot carries in its jar.
*   `max-jar-level`: with one parameter (`robot`), it represents the maximum amount of water (real number) that the robot can carry in its jar.
*   `tank-level`: with one parameter (`location`), it models the amount of water (real number) contained in the tank located at the given location.
*   `tank-goal`: with one parameter (`location`), it represents the desired amount of water (real number) for the tank located at that location.

In [ ]:
p = Problem('WaterTanks')

# Boolean fluents
at_robot = up.model.Fluent('at-robot', BoolType(), r=robot, l=location)
at_station = up.model.Fluent('at-station', BoolType(), l=location)
attached_to_station = up.model.Fluent('attached-to-station', BoolType(), r=robot, l=location)
at_tap = up.model.Fluent('at-tap', BoolType(), l=location)
at_tank = up.model.Fluent('at-tank', BoolType(), l=location)
tank_ready = up.model.Fluent('tank-ready', BoolType(), l=location)
p.add_fluent(at_robot, default_initial_value=False)
p.add_fluent(at_station, default_initial_value=False)
p.add_fluent(at_tap, default_initial_value=False)
p.add_fluent(at_tank, default_initial_value=False)
p.add_fluent(attached_to_station, default_initial_value=False)
p.add_fluent(tank_ready, default_initial_value=False)

# Numeric fluents
battery_level = up.model.Fluent('battery-level', RealType(), r=robot)
jar_level = up.model.Fluent('jar-level', RealType(), r=robot)
max_jar_level = up.model.Fluent('max-jar-level', RealType(), r=robot)
tank_level = up.model.Fluent('tank-level', RealType(), l=location)
tank_goal = up.model.Fluent('tank-goal', RealType(), l=location)
p.add_fluent(battery_level)
p.add_fluent(jar_level)
p.add_fluent(max_jar_level)
p.add_fluent(tank_level)
p.add_fluent(tank_goal)

## Action `move`

Using a `move` action, a robot can move from one location to another. The **duration of this action depends on the weight transported**, that is, on the amount of water the robot carries in its jar: `10 + jar-level(r)/10`. This action consumes 10% of the battery.

In [6]:
move = up.model.DurativeAction('move', r=robot, l1=location, l2=location)
r = move.parameter('r')
l1 = move.parameter('l1')
l2 = move.parameter('l2')
move.set_fixed_duration(Plus(10, Div(jar_level(r), 10)))
move.add_condition(StartTiming(), at_robot(r, l1))
move.add_condition(StartTiming(), GE(battery_level(r), 10))
move.add_effect(StartTiming(), at_robot(r, l1), False)
move.add_effect(EndTiming(), at_robot(r, l2), True)
p.add_action(move)

## Action `attach`

Using an `attach` action, a robot can dock to the charging station. To do this, it is a condition that the station is free (it can only be used by one robot at the same time). We model this using a **Forall** expression.
The planner is in charge of deciding how long the robot will remain anchored to the station (between 1 and 100 minutes). This is represented by a **duration interval**.

In [7]:
attach = up.model.DurativeAction('attach', r=robot, l=location)
r = attach.parameter('r')
l = attach.parameter('l')
rob = Variable('rob', robot)
attach.set_closed_duration_interval(1, 100)
attach.add_condition(StartTiming(), at_station(l))
attach.add_condition(StartTiming(), Forall(Not(attached_to_station(rob, l)), rob))
attach.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_robot(r, l))
attach.add_effect(StartTiming(), attached_to_station(r, l), True)
attach.add_effect(EndTiming(), attached_to_station(r, l), False)
p.add_action(attach)

## Action `charge`

Using a `charge` action, a robot can recharge its battery. The third parameter of the action is a **numeric parameter**, which allows the planner to choose the percentage of battery to recharge.

The duration of this action is a **non-linear expression**, which tries to model the fact that batteries charge slower as they get full.

In [8]:
charge = up.model.DurativeAction('charge', r=robot, l=location, percentage=RealType())
r = charge.parameter('r')
l = charge.parameter('l')
percentage = charge.parameter('percentage')
charge.set_fixed_duration(Plus(percentage,
                               Div(Times(battery_level(r), battery_level(r), percentage), 5000)))
charge.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), attached_to_station(r, l))
charge.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_robot(r, l))
charge.add_condition(StartTiming(), LT(battery_level(r), 100))
charge.add_condition(StartTiming(), LE(percentage, Minus(100, battery_level(r))))
charge.add_condition(StartTiming(), GT(percentage, 0))
charge.add_increase_effect(EndTiming(), battery_level(r), percentage)
p.add_action(charge)

## Action `fill-jar`

Using a `fill-jar` action, a robot can fill its jar. The third parameter of the action is a **numeric parameter**, which allows the planner to choose how much water to pour into the jar. The duration of this action is directly proportional to the amount of water poured. In any case, this action consumes 5% battery.

In [9]:
fill = up.model.DurativeAction('fill-jar', r=robot, l=location, amount=RealType())
r = fill.parameter('r')
l = fill.parameter('l')
amount = fill.parameter('amount')
fill.set_fixed_duration(amount)
fill.add_condition(StartTiming(), GE(amount, 1))
fill.add_condition(StartTiming(), LE(amount, Minus(max_jar_level(r), jar_level(r))))
fill.add_condition(StartTiming(), GE(battery_level(r), 5))
fill.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_tap(l))
fill.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_robot(r, l))
fill.add_condition(StartTiming(), at_station(l))
fill.add_decrease_effect(EndTiming(), battery_level(r), 5)
fill.add_increase_effect(EndTiming(), jar_level(r), amount)
p.add_action(fill)

## Action `empty-jar`

Using an `empty-jar` action, a robot can pour all the water it carries in its jar into a tank. The duration of this action is directly proportional to the amount of water poured. This action leaves the jar empty while increasing the level of the tank with the amount poured. This action consumes 5% battery.

In [10]:
empty = up.model.DurativeAction('empty-jar', r=robot, l=location)
r = empty.parameter('r')
l = empty.parameter('l')
empty.set_fixed_duration(jar_level(r))
empty.add_condition(StartTiming(), GE(battery_level(r), 5))
empty.add_condition(StartTiming(), LE(Plus(jar_level(r), tank_level(l)), 100))
empty.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_tank(l))
empty.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_robot(r, l))
empty.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), Not(tank_ready(l)))
empty.add_decrease_effect(EndTiming(), battery_level(r), 5)
empty.add_increase_effect(EndTiming(), tank_level(l), jar_level(r))
empty.add_effect(EndTiming(), jar_level(r), 0)
p.add_action(empty)

## Action `close-tank`

Using a `close-tank` action, a robot can close a tank when the water it contains has reached the desired level. This action consumes 5% battery.

In [11]:
close = up.model.DurativeAction('close-tank', r=robot, l=location)
r = close.parameter('r')
l = close.parameter('l')
close.set_fixed_duration(jar_level(r))
close.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_tank(l))
close.add_condition(StartTiming(), Not(tank_ready(l)))
close.add_condition(StartTiming(), Equals(tank_level(l), tank_goal(l)))
close.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), at_robot(r, l))
close.add_effect(StartTiming(), tank_ready(l), True)
p.add_action(close)

## Objects

We are now going to create a simple problem in which we will define a robot (`r1`) and two locations (`a` and `b`).

In [ ]:
r1 = Object('r1', robot)
a = Object('a', location)
b = Object('b', location)
p.add_object(r1)
p.add_object(a)
p.add_object(b)

## Initial state

In our example problem, we will have a charging station at location `a`, but it will only be available 10 minutes after the plan starts. We model this using a TIL (**timed initial literal**).

We will also have a tap at location `a` and two water tanks at `a` and `b`.

The robot will initially be in location `a` with a battery level of 15%. Its jar will be filled to the maximum (20 liters).

Both water tanks, in locations `a` and `b`, will be initially empty. The goal is for both tanks to contain 10 and 40 liters of water, respectively.

In [13]:
p.add_timed_effect(Timing(10, Timepoint(TimepointKind.GLOBAL_START)), at_station(a), True)
p.set_initial_value(at_tap(a), True)
p.set_initial_value(at_tank(a), True)
p.set_initial_value(at_tank(b), True)
p.set_initial_value(at_robot(r1, a), True)
p.set_initial_value(battery_level(r1), 15)
p.set_initial_value(jar_level(r1), 20)
p.set_initial_value(max_jar_level(r1), 20)
p.set_initial_value(tank_level(a), 0)
p.set_initial_value(tank_goal(a), 10)
p.set_initial_value(tank_level(b), 0)
p.set_initial_value(tank_goal(b), 40)

## Goal

Finally, the objective is for both tanks to be ready, that is, for the robot to close them once they have reached their desired level.

In [14]:
p.add_goal(tank_ready(a))
p.add_goal(tank_ready(b))

## Solving the problem

We now select the NextFLAP planner to solve the problem we have defined and show the plan obtained.

In [ ]:
with env.factory.OneshotPlanner(name='nextflap') as planner:
    result = planner.solve(p, timeout=60.0)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'; {planner.name} found a plan!')
        if result.plan.kind == up.plans.plan.PlanKind.PARTIAL_ORDER_PLAN:
            seqPlan = result.plan.convert_to(up.plans.plan.PlanKind.SEQUENTIAL_PLAN, p)
            for action in seqPlan.actions:
                print(action)
        else:
            for time, action, dur in result.plan.timed_actions:
                print(f'{time.numerator/time.denominator:.3f}: {action} [{dur.numerator/dur.denominator:.3f}]')
    else:
        print('No plan found!')